# Experiment 002: Multi-Phase Optimization with Backward Propagation

This experiment implements:
1. Enhanced C++ optimizer with higher iterations and multiple seeds
2. Backward propagation to improve smaller N configurations
3. Fractional translation for fine-grained polish
4. Multiple optimization passes

**Target:** Score < 80 (from current 135.82)

In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
from shapely.ops import unary_union
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import os
import subprocess
import shutil
import time

getcontext().prec = 25
scale_factor = Decimal("1e18")

print("Libraries loaded")

Libraries loaded


In [2]:
# ChristmasTree class
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
# Scoring and validation functions
def load_configuration_from_df(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])[1:] if str(row["x"]).startswith('s') else str(row["x"])
        y = str(row["y"])[1:] if str(row["y"]).startswith('s') else str(row["y"])
        deg = str(row["deg"])[1:] if str(row["deg"]).startswith('s') else str(row["deg"])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

def get_side_length(trees):
    if not trees:
        return 0.0
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    return max(max_x - min_x, max_y - min_y)

def get_score(trees, n):
    if not trees:
        return 0.0
    side = get_side_length(trees)
    return side**2 / n

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def score_submission(file_path, max_n=200):
    df = pd.read_csv(file_path)
    total_score = 0.0
    overlaps = []
    for n in range(1, max_n + 1):
        trees = load_configuration_from_df(n, df)
        if trees:
            total_score += get_score(trees, n)
            if has_overlap(trees):
                overlaps.append(n)
    return total_score, overlaps

print("Scoring functions defined")

Scoring functions defined


In [4]:
# Enhanced C++ optimizer with fractional translation
cpp_code = '''// Enhanced Tree Packer with Fractional Translation
#include <bits/stdc++.h>
#include <omp.h>
using namespace std;

constexpr int MAX_N = 200;
constexpr int NV = 15;
constexpr double PI = 3.14159265358979323846;

alignas(64) const double TX[NV] = {0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125};
alignas(64) const double TY[NV] = {0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5};

struct FastRNG {
    uint64_t s[2];
    FastRNG(uint64_t seed = 42) {
        s[0] = seed ^ 0x853c49e6748fea9bULL;
        s[1] = (seed * 0x9e3779b97f4a7c15ULL) ^ 0xc4ceb9fe1a85ec53ULL;
    }
    inline uint64_t rotl(uint64_t x, int k) { return (x << k) | (x >> (64 - k)); }
    inline uint64_t next() {
        uint64_t s0 = s[0], s1 = s[1], r = s0 + s1;
        s1 ^= s0; s[0] = rotl(s0, 24) ^ s1 ^ (s1 << 16); s[1] = rotl(s1, 37);
        return r;
    }
    inline double rf() { return (next() >> 11) * 0x1.0p-53; }
    inline double rf2() { return rf() * 2.0 - 1.0; }
    inline int ri(int n) { return next() % n; }
    inline double gaussian() {
        double u1 = rf() + 1e-10, u2 = rf();
        return sqrt(-2.0 * log(u1)) * cos(2.0 * PI * u2);
    }
};

struct Poly {
    double px[NV], py[NV];
    double x0, y0, x1, y1;
};

inline void getPoly(double cx, double cy, double deg, Poly& q) {
    double rad = deg * (PI / 180.0);
    double s = sin(rad), c = cos(rad);
    double minx = 1e9, miny = 1e9, maxx = -1e9, maxy = -1e9;
    for (int i = 0; i < NV; i++) {
        double x = TX[i] * c - TY[i] * s + cx;
        double y = TX[i] * s + TY[i] * c + cy;
        q.px[i] = x; q.py[i] = y;
        if (x < minx) minx = x; if (x > maxx) maxx = x;
        if (y < miny) miny = y; if (y > maxy) maxy = y;
    }
    q.x0 = minx; q.y0 = miny; q.x1 = maxx; q.y1 = maxy;
}

inline bool pip(double px, double py, const Poly& q) {
    bool in = false;
    int j = NV - 1;
    for (int i = 0; i < NV; i++) {
        if ((q.py[i] > py) != (q.py[j] > py) &&
            px < (q.px[j] - q.px[i]) * (py - q.py[i]) / (q.py[j] - q.py[i]) + q.px[i])
            in = !in;
        j = i;
    }
    return in;
}

inline bool segInt(double ax, double ay, double bx, double by,
                   double cx, double cy, double dx, double dy) {
    double d1 = (dx-cx)*(ay-cy) - (dy-cy)*(ax-cx);
    double d2 = (dx-cx)*(by-cy) - (dy-cy)*(bx-cx);
    double d3 = (bx-ax)*(cy-ay) - (by-ay)*(cx-ax);
    double d4 = (bx-ax)*(dy-ay) - (by-ay)*(dx-ax);
    return ((d1 > 0) != (d2 > 0)) && ((d3 > 0) != (d4 > 0));
}

inline bool overlap(const Poly& a, const Poly& b) {
    if (a.x1 < b.x0 || b.x1 < a.x0 || a.y1 < b.y0 || b.y1 < a.y0) return false;
    for (int i = 0; i < NV; i++) {
        if (pip(a.px[i], a.py[i], b)) return true;
        if (pip(b.px[i], b.py[i], a)) return true;
    }
    for (int i = 0; i < NV; i++) {
        int ni = (i + 1) % NV;
        for (int j = 0; j < NV; j++) {
            int nj = (j + 1) % NV;
            if (segInt(a.px[i], a.py[i], a.px[ni], a.py[ni],
                      b.px[j], b.py[j], b.px[nj], b.py[nj])) return true;
        }
    }
    return false;
}

struct Cfg {
    int n;
    double x[MAX_N], y[MAX_N], a[MAX_N];
    Poly pl[MAX_N];
    double gx0, gy0, gx1, gy1;

    inline void upd(int i) { getPoly(x[i], y[i], a[i], pl[i]); }
    inline void updAll() { for (int i = 0; i < n; i++) upd(i); updGlobal(); }
    inline void updGlobal() {
        gx0 = gy0 = 1e9; gx1 = gy1 = -1e9;
        for (int i = 0; i < n; i++) {
            if (pl[i].x0 < gx0) gx0 = pl[i].x0;
            if (pl[i].x1 > gx1) gx1 = pl[i].x1;
            if (pl[i].y0 < gy0) gy0 = pl[i].y0;
            if (pl[i].y1 > gy1) gy1 = pl[i].y1;
        }
    }
    inline bool hasOvl(int i) const {
        for (int j = 0; j < n; j++)
            if (i != j && overlap(pl[i], pl[j])) return true;
        return false;
    }
    inline bool anyOvl() const {
        for (int i = 0; i < n; i++)
            for (int j = i + 1; j < n; j++)
                if (overlap(pl[i], pl[j])) return true;
        return false;
    }
    inline double side() const { return max(gx1 - gx0, gy1 - gy0); }
    inline double score() const { double s = side(); return s * s / n; }
};

Cfg squeeze(Cfg c) {
    double cx = (c.gx0 + c.gx1) / 2.0, cy = (c.gy0 + c.gy1) / 2.0;
    for (double scale = 0.9995; scale >= 0.98; scale -= 0.0005) {
        Cfg trial = c;
        for (int i = 0; i < c.n; i++) {
            trial.x[i] = cx + (c.x[i] - cx) * scale;
            trial.y[i] = cy + (c.y[i] - cy) * scale;
        }
        trial.updAll();
        if (!trial.anyOvl()) c = trial;
        else break;
    }
    return c;
}

Cfg compaction(Cfg c, int iters) {
    double bs = c.side();
    for (int it = 0; it < iters; it++) {
        double cx = (c.gx0 + c.gx1) / 2.0, cy = (c.gy0 + c.gy1) / 2.0;
        bool improved = false;
        for (int i = 0; i < c.n; i++) {
            double ox = c.x[i], oy = c.y[i];
            double dx = cx - c.x[i], dy = cy - c.y[i];
            double d = sqrt(dx*dx + dy*dy);
            if (d < 1e-6) continue;
            for (double step : {0.02, 0.008, 0.003, 0.001, 0.0004}) {
                c.x[i] = ox + dx/d * step; c.y[i] = oy + dy/d * step; c.upd(i);
                if (!c.hasOvl(i)) {
                    c.updGlobal();
                    if (c.side() < bs - 1e-12) { bs = c.side(); improved = true; ox = c.x[i]; oy = c.y[i]; }
                    else { c.x[i] = ox; c.y[i] = oy; c.upd(i); }
                } else { c.x[i] = ox; c.y[i] = oy; c.upd(i); }
            }
        }
        c.updGlobal();
        if (!improved) break;
    }
    return c;
}

Cfg localSearch(Cfg c, int maxIter) {
    double bs = c.side();
    const double steps[] = {0.01, 0.004, 0.0015, 0.0006, 0.00025, 0.0001};
    const double rots[] = {5.0, 2.0, 0.8, 0.3, 0.1};
    const int dx[] = {1,-1,0,0,1,1,-1,-1};
    const int dy[] = {0,0,1,-1,1,-1,1,-1};

    for (int iter = 0; iter < maxIter; iter++) {
        bool improved = false;
        for (int i = 0; i < c.n; i++) {
            double cx = (c.gx0 + c.gx1) / 2.0, cy = (c.gy0 + c.gy1) / 2.0;
            double ddx = cx - c.x[i], ddy = cy - c.y[i];
            double dist = sqrt(ddx*ddx + ddy*ddy);
            if (dist > 1e-6) {
                for (double st : steps) {
                    double ox = c.x[i], oy = c.y[i];
                    c.x[i] += ddx/dist * st; c.y[i] += ddy/dist * st; c.upd(i);
                    if (!c.hasOvl(i)) { c.updGlobal(); if (c.side() < bs - 1e-12) { bs = c.side(); improved = true; }
                        else { c.x[i]=ox; c.y[i]=oy; c.upd(i); c.updGlobal(); } }
                    else { c.x[i]=ox; c.y[i]=oy; c.upd(i); }
                }
            }
            for (double st : steps) {
                for (int d = 0; d < 8; d++) {
                    double ox=c.x[i], oy=c.y[i];
                    c.x[i] += dx[d]*st; c.y[i] += dy[d]*st; c.upd(i);
                    if (!c.hasOvl(i)) { c.updGlobal(); if (c.side() < bs - 1e-12) { bs = c.side(); improved = true; }
                        else { c.x[i]=ox; c.y[i]=oy; c.upd(i); c.updGlobal(); } }
                    else { c.x[i]=ox; c.y[i]=oy; c.upd(i); }
                }
            }
            for (double rt : rots) {
                for (double da : {rt, -rt}) {
                    double oa = c.a[i]; c.a[i] += da;
                    while (c.a[i] < 0) c.a[i] += 360; while (c.a[i] >= 360) c.a[i] -= 360;
                    c.upd(i);
                    if (!c.hasOvl(i)) { c.updGlobal(); if (c.side() < bs - 1e-12) { bs = c.side(); improved = true; }
                        else { c.a[i]=oa; c.upd(i); c.updGlobal(); } }
                    else { c.a[i]=oa; c.upd(i); }
                }
            }
        }
        if (!improved) break;
    }
    return c;
}

Cfg fractionalTranslation(Cfg c, int maxIter = 200) {
    Cfg best = c;
    double bs = best.side();
    double frac_steps[] = {0.001, 0.0005, 0.0002, 0.0001, 0.00005, 0.00002, 0.00001};
    int dx[] = {0, 0, 1, -1, 1, 1, -1, -1};
    int dy[] = {1, -1, 0, 0, 1, -1, 1, -1};
    
    for (int iter = 0; iter < maxIter; iter++) {
        bool improved = false;
        for (int i = 0; i < c.n; i++) {
            for (double step : frac_steps) {
                for (int d = 0; d < 8; d++) {
                    double ox = best.x[i], oy = best.y[i];
                    best.x[i] += dx[d] * step;
                    best.y[i] += dy[d] * step;
                    best.upd(i);
                    if (!best.hasOvl(i)) {
                        best.updGlobal();
                        double ns = best.side();
                        if (ns < bs - 1e-12) {
                            bs = ns;
                            improved = true;
                        } else {
                            best.x[i] = ox; best.y[i] = oy; best.upd(i);
                        }
                    } else {
                        best.x[i] = ox; best.y[i] = oy; best.upd(i);
                    }
                }
            }
        }
        if (!improved) break;
    }
    return best;
}

Cfg simulatedAnnealing(Cfg c, int maxIter, FastRNG& rng, double Tmax=1.0, double Tmin=0.000005, double alpha=0.25) {
    double T = Tmax;
    double bs = c.side();
    Cfg best = c;
    
    for (int iter = 0; iter < maxIter && T > Tmin; iter++) {
        int i = rng.ri(c.n);
        double ox = c.x[i], oy = c.y[i], oa = c.a[i];
        
        int moveType = rng.ri(3);
        if (moveType == 0) {
            c.x[i] += rng.gaussian() * 0.02 * T;
            c.y[i] += rng.gaussian() * 0.02 * T;
        } else if (moveType == 1) {
            c.a[i] += rng.gaussian() * 5.0 * T;
            while (c.a[i] < 0) c.a[i] += 360; while (c.a[i] >= 360) c.a[i] -= 360;
        } else {
            double cx = (c.gx0 + c.gx1) / 2.0, cy = (c.gy0 + c.gy1) / 2.0;
            double dx = cx - c.x[i], dy = cy - c.y[i];
            double d = sqrt(dx*dx + dy*dy);
            if (d > 1e-6) {
                c.x[i] += dx/d * 0.01 * rng.rf();
                c.y[i] += dy/d * 0.01 * rng.rf();
            }
        }
        c.upd(i);
        
        if (c.hasOvl(i)) {
            c.x[i] = ox; c.y[i] = oy; c.a[i] = oa; c.upd(i);
        } else {
            c.updGlobal();
            double ns = c.side();
            if (ns < bs) {
                bs = ns;
                best = c;
            } else if (rng.rf() < exp((bs - ns) / T)) {
                // Accept worse
            } else {
                c.x[i] = ox; c.y[i] = oy; c.a[i] = oa; c.upd(i); c.updGlobal();
            }
        }
        T *= (1.0 - alpha / maxIter);
    }
    return best;
}

Cfg configs[MAX_N + 1];
double best_sides[MAX_N + 1];

void parse_csv(const string& filename) {
    ifstream f(filename);
    string line;
    getline(f, line);
    
    for (int n = 1; n <= MAX_N; n++) {
        configs[n].n = n;
        best_sides[n] = 1e9;
    }
    
    while (getline(f, line)) {
        size_t p1 = line.find(',');
        size_t p2 = line.find(',', p1+1);
        size_t p3 = line.find(',', p2+1);
        
        string id = line.substr(0, p1);
        string xs = line.substr(p1+1, p2-p1-1);
        string ys = line.substr(p2+1, p3-p2-1);
        string ds = line.substr(p3+1);
        
        if (xs[0] == 's') xs = xs.substr(1);
        if (ys[0] == 's') ys = ys.substr(1);
        if (ds[0] == 's') ds = ds.substr(1);
        
        int n = stoi(id.substr(0, 3));
        int idx = stoi(id.substr(4));
        
        configs[n].x[idx] = stod(xs);
        configs[n].y[idx] = stod(ys);
        configs[n].a[idx] = stod(ds);
    }
    
    for (int n = 1; n <= MAX_N; n++) {
        configs[n].updAll();
        best_sides[n] = configs[n].side();
    }
}

void save_csv(const string& filename) {
    ofstream f(filename);
    f << fixed << setprecision(15);
    f << "id,x,y,deg\\n";
    
    for (int n = 1; n <= MAX_N; n++) {
        for (int i = 0; i < n; i++) {
            f << setfill('0') << setw(3) << n << "_" << i << ",";
            f << "s" << configs[n].x[i] << ",";
            f << "s" << configs[n].y[i] << ",";
            f << "s" << configs[n].a[i] << "\\n";
        }
    }
}

double calc_total_score() {
    double total = 0;
    for (int n = 1; n <= MAX_N; n++) {
        total += best_sides[n] * best_sides[n] / n;
    }
    return total;
}

int main(int argc, char** argv) {
    int n_iters = 5000;
    int n_rounds = 48;
    int seed_offset = 0;
    
    for (int i = 1; i < argc; i++) {
        if (string(argv[i]) == "-n" && i+1 < argc) n_iters = stoi(argv[++i]);
        if (string(argv[i]) == "-r" && i+1 < argc) n_rounds = stoi(argv[++i]);
        if (string(argv[i]) == "-s" && i+1 < argc) seed_offset = stoi(argv[++i]);
    }
    
    cout << "Enhanced Tree Packer with Fractional Translation" << endl;
    cout << "Iterations: " << n_iters << ", Rounds: " << n_rounds << ", Seed: " << seed_offset << endl;
    
    parse_csv("submission.csv");
    cout << fixed << setprecision(6);
    cout << "Initial: " << calc_total_score() << endl;
    
    #pragma omp parallel for schedule(dynamic)
    for (int n = 1; n <= MAX_N; n++) {
        FastRNG rng(42 + n + seed_offset * 1000);
        Cfg best = configs[n];
        double best_side = best.side();
        
        for (int r = 0; r < n_rounds; r++) {
            Cfg c = best;
            
            // SA with higher temperature
            c = simulatedAnnealing(c, n_iters, rng, 1.0, 0.000005, 0.25);
            
            // Local search
            c = localSearch(c, 100);
            
            // Compaction
            c = compaction(c, 30);
            
            // Squeeze
            c = squeeze(c);
            
            // Fractional translation
            c = fractionalTranslation(c, 100);
            
            if (c.side() < best_side && !c.anyOvl()) {
                best_side = c.side();
                best = c;
            }
        }
        
        #pragma omp critical
        {
            if (best_side < best_sides[n]) {
                configs[n] = best;
                best_sides[n] = best_side;
            }
        }
    }
    
    cout << "Final:   " << calc_total_score() << endl;
    save_csv("submission_optimized.csv");
    
    return 0;
}
'''

with open('/home/code/experiments/002_multiphase/tree_packer_v2.cpp', 'w') as f:
    f.write(cpp_code)

print("Enhanced C++ optimizer written")

Enhanced C++ optimizer written


In [5]:
# Compile the enhanced optimizer
os.chdir('/home/code/experiments/002_multiphase')

result = subprocess.run(
    ['g++', '-O3', '-march=native', '-std=c++17', '-fopenmp', '-o', 'tree_packer_v2', 'tree_packer_v2.cpp'],
    capture_output=True, text=True
)

if result.returncode == 0:
    print("Compilation successful!")
else:
    print(f"Compilation failed:\n{result.stderr}")

Compilation successful!


In [6]:
# Copy the best submission from experiment 001 as starting point
shutil.copy('/home/code/experiments/001_baseline/submission_final.csv', 
            '/home/code/experiments/002_multiphase/submission.csv')

# Verify starting score
start_score, overlaps = score_submission('/home/code/experiments/002_multiphase/submission.csv')
print(f"Starting score: {start_score:.6f}")
print(f"Overlaps: {overlaps}")

Starting score: 135.819103
Overlaps: []


In [ ]:
# Phase 1: Run enhanced optimizer with multiple seeds
print("=" * 60)
print("PHASE 1: Multi-seed optimization")
print("=" * 60)

best_score = start_score
best_file = '/home/code/experiments/002_multiphase/submission.csv'

for seed in range(3):  # 3 different seeds
    print(f"\n--- Seed {seed} ---")
    
    # Copy current best to working file
    shutil.copy(best_file, '/home/code/experiments/002_multiphase/submission.csv')
    
    start_time = time.time()
    result = subprocess.run(
        ['./tree_packer_v2', '-n', '5000', '-r', '48', '-s', str(seed)],
        capture_output=True, text=True,
        cwd='/home/code/experiments/002_multiphase'
    )
    elapsed = time.time() - start_time
    
    print(f"Completed in {elapsed:.1f}s")
    print(result.stdout)
    
    # Check if improved
    if os.path.exists('/home/code/experiments/002_multiphase/submission_optimized.csv'):
        new_score, overlaps = score_submission('/home/code/experiments/002_multiphase/submission_optimized.csv')
        print(f"Score: {new_score:.6f}, Overlaps: {len(overlaps)}")
        
        if new_score < best_score and len(overlaps) == 0:
            best_score = new_score
            shutil.copy('/home/code/experiments/002_multiphase/submission_optimized.csv',
                       f'/home/code/experiments/002_multiphase/best_seed{seed}.csv')
            best_file = f'/home/code/experiments/002_multiphase/best_seed{seed}.csv'
            print(f"NEW BEST: {best_score:.6f}")

print(f"\nPhase 1 best score: {best_score:.6f}")

In [ ]:
# Phase 2: Backward Propagation
print("\n" + "=" * 60)
print("PHASE 2: Backward Propagation")
print("=" * 60)

def backward_propagation(input_file, output_file):
    """Apply backward propagation: use larger N configs to improve smaller N."""
    df = pd.read_csv(input_file)
    
    # Load all configurations
    configs = {}
    sides = {}
    
    for n in range(1, 201):
        trees = load_configuration_from_df(n, df)
        if trees:
            configs[n] = trees
            sides[n] = get_side_length(trees)
    
    print(f"Initial total score: {sum(s**2/n for n, s in sides.items()):.6f}")
    
    improvements = 0
    
    # Backward iteration from N=200 to N=2
    for n in range(200, 1, -1):
        if n not in configs or (n-1) not in configs:
            continue
            
        current_side = sides[n-1]
        best_side = current_side
        best_tree_to_remove = None
        
        # Try removing each tree from config[n]
        for tree_idx in range(n):
            # Create candidate by removing tree at tree_idx
            candidate_trees = [t for i, t in enumerate(configs[n]) if i != tree_idx]
            
            if len(candidate_trees) != n - 1:
                continue
                
            candidate_side = get_side_length(candidate_trees)
            
            # Check if this is better and has no overlaps
            if candidate_side < best_side and not has_overlap(candidate_trees):
                best_side = candidate_side
                best_tree_to_remove = tree_idx
        
        # If we found an improvement, apply it
        if best_tree_to_remove is not None:
            new_trees = [t for i, t in enumerate(configs[n]) if i != best_tree_to_remove]
            configs[n-1] = new_trees
            old_side = sides[n-1]
            sides[n-1] = best_side
            improvements += 1
            if improvements <= 10:  # Print first 10 improvements
                print(f"  N={n-1}: {old_side:.6f} -> {best_side:.6f} (from N={n})")
    
    print(f"Total improvements: {improvements}")
    
    # Save the result
    rows = []
    for n in range(1, 201):
        if n in configs:
            for i, tree in enumerate(configs[n]):
                rows.append({
                    'id': f"{n:03d}_{i}",
                    'x': f"s{float(tree.center_x)}",
                    'y': f"s{float(tree.center_y)}",
                    'deg': f"s{float(tree.angle)}"
                })
    
    result_df = pd.DataFrame(rows)
    result_df.to_csv(output_file, index=False)
    
    final_score = sum(s**2/n for n, s in sides.items())
    print(f"Final total score: {final_score:.6f}")
    
    return final_score

# Apply backward propagation
bp_score = backward_propagation(best_file, '/home/code/experiments/002_multiphase/submission_bp.csv')

if bp_score < best_score:
    best_score = bp_score
    best_file = '/home/code/experiments/002_multiphase/submission_bp.csv'
    print(f"Backward propagation improved score to: {best_score:.6f}")

In [ ]:
# Phase 3: Fix Direction (rotation optimization)
print("\n" + "=" * 60)
print("PHASE 3: Fix Direction (Rotation Optimization)")
print("=" * 60)

def calculate_bbox_side_at_angle(angle_deg, points):
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees):
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points) / float(scale_factor)
    
    try:
        hull_points = points_np[ConvexHull(points_np).vertices]
    except:
        return get_side_length(trees), 0.0
    
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    res = minimize_scalar(lambda a: calculate_bbox_side_at_angle(a, hull_points),
                          bounds=(0.001, 89.999), method='bounded')
    
    if res.fun < initial_side - 1e-8:
        return res.fun, res.x
    return initial_side, 0.0

def apply_rotation(trees, angle_deg):
    if not trees or abs(angle_deg) < 1e-9:
        return trees
    
    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0]) / float(scale_factor)
    
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])
    
    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center
    rotated = shifted.dot(rot_matrix.T) + rotation_center
    
    rotated_trees = []
    for i in range(len(trees)):
        new_tree = ChristmasTree(
            str(rotated[i, 0]), 
            str(rotated[i, 1]),
            str(float(trees[i].angle) + angle_deg)
        )
        rotated_trees.append(new_tree)
    return rotated_trees

def fix_direction(input_path, output_path):
    df = pd.read_csv(input_path)
    
    configs = {}
    sides = {}
    
    for n in range(1, 201):
        trees = load_configuration_from_df(n, df)
        if trees:
            configs[n] = trees
            sides[n] = get_side_length(trees)
    
    initial_score = sum(s**2/n for n, s in sides.items())
    print(f"Initial score: {initial_score:.6f}")
    
    improved_count = 0
    for n in range(1, 201):
        if n not in configs or len(configs[n]) < 2:
            continue
            
        trees = configs[n]
        try:
            best_side, best_angle = optimize_rotation(trees)
            if abs(best_angle) > 0.001 and best_side < sides[n] - 1e-8:
                rotated_trees = apply_rotation(trees, best_angle)
                if not has_overlap(rotated_trees):
                    configs[n] = rotated_trees
                    sides[n] = best_side
                    improved_count += 1
        except:
            pass
    
    print(f"Improved {improved_count} groups")
    
    # Save
    rows = []
    for n in range(1, 201):
        if n in configs:
            for i, tree in enumerate(configs[n]):
                rows.append({
                    'id': f"{n:03d}_{i}",
                    'x': f"s{float(tree.center_x)}",
                    'y': f"s{float(tree.center_y)}",
                    'deg': f"s{float(tree.angle)}"
                })
    
    result_df = pd.DataFrame(rows)
    result_df.to_csv(output_path, index=False)
    
    final_score = sum(s**2/n for n, s in sides.items())
    print(f"Final score: {final_score:.6f}")
    return final_score

fd_score = fix_direction(best_file, '/home/code/experiments/002_multiphase/submission_fd.csv')

if fd_score < best_score:
    best_score = fd_score
    best_file = '/home/code/experiments/002_multiphase/submission_fd.csv'
    print(f"Fix direction improved score to: {best_score:.6f}")

In [ ]:
# Final validation
print("\n" + "=" * 60)
print("FINAL VALIDATION")
print("=" * 60)

final_score, overlaps = score_submission(best_file)
print(f"Final score: {final_score:.6f}")
print(f"Overlaps: {overlaps}")

if len(overlaps) == 0:
    print("\n✅ Validation SUCCESSFUL - No overlaps")
else:
    print(f"\n❌ Validation FAILED - {len(overlaps)} overlapping configurations")

In [ ]:
# Copy to submission folder
shutil.copy(best_file, '/home/submission/submission.csv')
print(f"Copied to /home/submission/submission.csv")

# Summary
print("\n" + "=" * 60)
print("EXPERIMENT SUMMARY")
print("=" * 60)
print(f"Starting score (from exp 001): {start_score:.6f}")
print(f"Final optimized score: {final_score:.6f}")
print(f"Improvement: {start_score - final_score:.6f}")
print(f"Target: 68.931058")
print(f"Gap to target: {final_score - 68.931058:.6f}")